# Imports

In [1]:
# Dependencies and Setup

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "booth_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Generate City List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

619

# Make Requests

In [5]:
#init lists to hold parsed data
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = cloudiness

df.head(10)

Got data for city index: 0 of 619
Missing data in OpenWeatherAPI for illoqqortoormiut
Got data for city index: 5 of 619
Got data for city index: 10 of 619
Got data for city index: 15 of 619
Got data for city index: 20 of 619
Got data for city index: 25 of 619
Got data for city index: 30 of 619
Got data for city index: 35 of 619
Got data for city index: 40 of 619
Got data for city index: 45 of 619
Got data for city index: 50 of 619
Got data for city index: 55 of 619
Got data for city index: 60 of 619
Missing data in OpenWeatherAPI for richard toll
Got data for city index: 65 of 619
Got data for city index: 70 of 619
Got data for city index: 75 of 619
Got data for city index: 80 of 619
Missing data in OpenWeatherAPI for lolua
Missing data in OpenWeatherAPI for amderma
Missing data in OpenWeatherAPI for ngukurr
Got data for city index: 85 of 619
Missing data in OpenWeatherAPI for saleaula
Got data for city index: 90 of 619
Missing data in OpenWeatherAPI for mys shmidta
Got data for city i

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Khani,41.96,42.96,64.40,60.37,63,1015,8.05,8.05
1,Marang,5.21,103.21,75.09,82.56,94,1010,3.36,3.36
2,Labuhan,-6.88,112.21,81.72,87.93,78,1012,6.51,6.51
3,Ushuaia,-54.80,-68.30,57.20,51.21,54,998,6.93,6.93
4,Lebu,-37.62,-73.65,50.86,46.35,78,1024,5.66,5.66
5,Bluff,-46.60,168.33,57.99,49.98,84,1006,15.99,15.99
6,Avarua,-21.21,-159.78,71.60,75.54,88,1015,4.70,4.70
7,Wattegama,6.80,81.48,74.77,82.62,90,1010,1.12,1.12
8,Rondonópolis,-16.47,-54.64,78.46,76.53,37,1010,3.58,3.58
9,Limbe,4.02,9.21,77.00,87.30,100,1012,2.24,2.24


In [7]:
df.to_csv(output_data_file, index=False)